In [1]:
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import time
import sys
import os

import seaborn as sns
import matplotlib.dates as mdates
from datetime import datetime, date

# preamble for jupyter notebook and django
import django
import platform

if platform.node() == "srv-mcc-apsis":
    sys.path.append("/home/muef/tmv/BasicBrowser/")
    #sys.path.append('/home/leey/tmv/BasicBrowser/')
    dtm_path = "/home/galm/software/dtm/dtm/main"
else:
    # local paths
    sys.path.append('/media/Data/MCC/tmv/BasicBrowser/')
    dtm_path = "/home/finn/dtm/dtm/main"

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
import parliament.models as pm
import scoping.models as sm
from parliament.tasks import do_search, run_tm, combine_searches
import cities.models as cmodels
from django.contrib.auth.models import User
from tmv_app.models import * 
from utils.tm_mgmt import update_topic_scores
from django.db.models import Q, Count, Func, F, Sum, Avg, Value as V
from django.db.models.functions import TruncDate, TruncMonth, TruncYear

In [2]:
# topic labeling

run_id = 1749
no_top_words = 20

labels = {}

input = False

for topic in Topic.objects.filter(run_id = run_id):
    stat = RunStats.objects.get(run_id=run_id)

    print("Topic #{}".format(topic.id))
    
    if stat.method == 'BD':
        
        terms = Term.objects.filter(topicterm__topic=topic).values('title').annotate(
    summed_score = Sum(F('topicterm__score'))).order_by('-summed_score')[:20]
        term_list = [t['title'] for t in terms]
        print(", ".join(term_list))

    else:
        print("Top words: {}".format(topic.top_words))

    if input:
        label = input("topic label: ")
        if label:
            labels[topic.id] = label
            topic.title = label
            topic.save()
    else:
        print("topic label: {}".format(topic.title))
    
    print("")

print(labels)

Topic #167068
prozent, energi, deutschland, erneuerbar, ziel, anteil, kohl, energiepolit, energietrag, fossil, strom, technologi, deutsch, hoh, klimaschutz, erreich, wirtschaft, kraftwerk, energieversorg, weltweit
topic label: targets

Topic #167069
energiew, energi, kost, erneuerbar, verbrauch, arbeitsplatz, eeg, unternehm, euro, strompreis, industri, preis, cent, milliard, deutschland, pro, kolleginn, zahl, strom, prozent
topic label: energy transition 3

Topic #167071
merkel, wirklich, red, endlich, einfach, eigent, geld, tun, zukunft, regier, mal, leb, wiss, sorg, find, vielleicht, lang, wenig, macht, warum
topic label: procedural 3

Topic #167072
gesetz, wettbewerb, gesetzentwurf, markt, strom, unternehm, verbrauch, kraftwarmekoppl, netz, bundesregier, preis, anlag, stadtwerk, strommarkt, kwk, bereich, kost, richtig, desweg, forder
topic label: electricity market

Topic #167073
international, china, global, entwicklungsland, deutsch, nachhalt, usa, indi, entwickl, staat, weltweit,

In [13]:
# modify single labels:

topic = Topic.objects.get(id=167080)
topic.title = 'mixed 3'
topic.save()